# Install Dependencies

In [ ]:
!pip install numpy==1.21.4
!pip install pandas==1.3.4
!pip install loguru==0.5.3

# Mount Clone Extract

**In case you have enough space on your disk, you could download full and updated database using `wget`, extract the database files and enjoy. Keep in mind, archived texts are prone to expand ~10x in size and if your archive is just 30 Gb, it's probably gonna take another 300 Gb after extraction. In any case, there's a nice small but sufficient chunk of data (November 2021) in the cloned repo.**

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
# replace with your path
%cd "drive/MyDrive/gar_colab"

In [ ]:
!git clone https://github.com/nurtdinovadf/garbdfias.git

In [ ]:
%cd garbdfias

In [ ]:
!ls

In [ ]:
!tar -xvf data.tar.gz

In [ ]:
!ls data

# Import

In [ ]:
import datetime
import gc
import glob
import os
import time
import xml.etree.ElementTree as ET

import numpy as np
import pandas as pd
from loguru import logger
from tqdm import tqdm

# Define Helper Functions

**These are the utilities I'm gonna use, it's just nice to have them all gathered in one place. I provided some short descriptions, and I might or might not explain what's going on a bit later.**

In [ ]:
def cleanup(x):
    """
    Manual object cleaning
    """
    del x
    gc.collect()


def parse_xml(x):
    """
    Parse GAR XML file into pandas dataframe object
    """
    tree = ET.parse(x)
    root = tree.getroot()
    df = [child.attrib for child in root]
    df = pd.DataFrame.from_dict(df)
    return df


def get_adms(df):
    """
    Get administrative "object-parent" relations into dictionary for later use
    """
    rftree = df[['OBJECTID', 'PARENTOBJID']].groupby(
        by='OBJECTID'
    )['PARENTOBJID'].apply(list).to_dict()
    return rftree


def get_adms_rec_rev(chain, rdadm, housdict, objdict):
    """
    Recursive address chain builder
    """
    objid = chain[-1]

    if objid in rdadm and objid == objid:
        prnts = rdadm[objid]
        if len(prnts) > 1:
            prnts = [x for x in prnts if x in objdict]
            if len(prnts) > 1:
                seedOKTMO = housdict[chain[0]]['OKTMO']
                prntOKTMO = {prnt: objdict[prnt]['OKTMO'] for prnt in prnts}
                prnts = [prnt for prnt in prnts if len(set(seedOKTMO).intersection(set(prntOKTMO[prnt]))) > 0]
            if len(prnts) == 0:
                return None
        chains = [chain + [obj] for obj in prnts if obj == obj]
        if len(chains) > 1:
            return [get_adms_rec_rev(ch, rdadm, housdict, objdict) for ch in chains]
        if len(chains) == 0:
            return chain
        return get_adms_rec_rev(chains[0], rdadm, housdict, objdict)
    else:
        return chain

    
def reduce_included(x):
    """
    Reduce included chains
    """
    maxl = 0
    maxch = []
    x = [tuple(y) for y in x]
    for y in x:
        maxl = (len(y) > maxl) * len(y) + (len(y) <= maxl) * maxl
        maxch = y if len(y) == maxl else maxch
    mask = [len(set(y).intersection(set(maxch))) == len(y) and y != maxch for y in x]
    ret = [y for y, z in zip(x, mask) if not z]
    single = False
    if len(ret) == 1:
        ret = ret[0]
        single = True
    return ret, single


def get_town(x):
    """
    Chain post-cleanup.
    """
    priority = ['5', '6', '4', '7', '1']
    street = [f'{i}' for i in range(8, 0, -1)]
    streets = [p for p in street if x[p] == 1]
    if len(streets) == 0:
        street = None
    else:
        street = streets[0]
    town = [p for p in priority if p != street and x[p] == 1]
    town = town[0] if len(town) > 0 else None
    leftover = [
        x for x in streets
        if x != street
        and x != town
        and x not in ['1', '2', '3']
    ]
    muni = [x for x in streets if x in ['2', '3']]

    return street, town, leftover, muni

# Core

## Define Region

**We'll be working on one particular region in this tutorial.**

In [ ]:
regions = glob.glob('data/[0-9][0-9]')
region = regions[0]
region

## Read Address Object File

**XML files in the DB are quite similar in structure, so we'll use `parse_xml` utility to convert XML files to pandas DataFrames.**

**Filter out historical data by `ISACTUAL` and `ISACTIVE` attributes.**

In [ ]:
fname = glob.glob(os.path.join(region, 'AS_ADDR_OBJ_*.XML'))
fname = [x for x in fname if 'PARAMS' not in x and 'DIVISION' not in x]
if len(fname) != 1:
    msg = f'Please check file count for region {region} there are {len(fname)} files'
    logger.error(msg)
    raise Exception(msg)
fname = fname[0]
adobj = parse_xml(fname)
adobj = adobj[(adobj['ISACTUAL'] == '1') & (adobj['ISACTIVE'] == '1')]
adobj.head()

**The way of joining address object types is a bit messed up, just like a lot of other stuff in here. We'll rename some columns right away as to not to string along same info in different columns.**

In [ ]:
fname = glob.glob('data/AS_ADDR_OBJ_TYPES_*.XML')
if len(fname) != 1:
    msg = f'Please check file count for region {region} there are {len(fname)} files'
    logger.error(msg)
    raise Exception(msg)
fname = fname[0]
adobjt = parse_xml(fname)
adobjt.head()

In [ ]:
adobj = adobj.merge(
    adobjt[['SHORTNAME', 'DESC', 'LEVEL']].rename(
        columns={
            'SHORTNAME': 'TYPENAME',
            'DESC': 'TYPELONGNAME'
        }
    ),
    on=['LEVEL', 'TYPENAME']
)
cleanup(adobjt)
adobj.head()

**Levels are not as strict as one might think and similar address objects ofthen end up on different levels.**

In [ ]:
fname = glob.glob('data/AS_OBJECT_LEVELS_*.XML')
if len(fname) != 1:
    msg = f'Please check file count for region {region} there are {len(fname)} files'
    logger.error(msg)
    raise Exception(msg)
fname = fname[0]
lev = parse_xml(fname)

adobj = adobj.merge(
    lev[['NAME', 'LEVEL']].rename(
        columns={
            'NAME': 'LEVELNAME'
        }
    ),
    on='LEVEL'
)
adobj.head()

**Some additional parameters, we'll need only `OKTMO` attribute, which is type 7 btw.**

In [ ]:
fname = glob.glob(os.path.join(region, 'AS_ADDR_OBJ_PARAMS_*.XML'))
fname = fname[0]
adobjp = parse_xml(fname)
adobjp.head()

In [ ]:
adobjp = adobjp[
    (
        adobjp['TYPEID'] == '7'
    ) & (
        adobjp['CHANGEIDEND'] == '0'
    )
]
adobjp = adobjp[
    adobjp.ENDDATE.apply(
        lambda x: datetime.datetime.strptime(
            x, '%Y-%m-%d'
        ) > datetime.datetime.fromtimestamp(
            time.time()
        )
    )
]
adobjp = adobjp[['OBJECTID', 'VALUE']].rename(
    columns={'VALUE': 'OKTMO'}
).drop_duplicates().groupby(
    by='OBJECTID'
).agg(
    lambda x: x.to_list()
).to_dict('index')
adobjp

## Read Houses File

**Reading houses data process is more or less the same.**

In [ ]:
fname = glob.glob(os.path.join(region, 'AS_HOUSES_*.XML'))
fname = [x for x in fname if 'PARAMS' not in x]
if len(fname) != 1:
    msg = f'Please check file count for region {region} there are {len(fname)} files'
    logger.error(msg)
    raise Exception(msg)
fname = fname[0]
hous = parse_xml(fname)
hous = hous.rename(
    columns={
        'ADDTYPE1': 'HOUSETYPE1',
        'ADDTYPE2': 'HOUSETYPE2',
        'ADDNUM1': 'HOUSENUM1',
        'ADDNUM2': 'HOUSENUM2'
    }
)
if 'ISACTUAL' in hous.columns:
    hous = hous[(hous['ISACTUAL'] == '1') & (hous['ISACTIVE'] == '1')]
else:
    hous = hous[(hous['ISACTIVE'] == '1')]
hous.head()

In [ ]:
fname = glob.glob('data/AS_HOUSE_TYPES_*.XML')
if len(fname) != 1:
    msg = f'Please check file count for region {region} there are {len(fname)} files'
    logger.error(msg)
    raise Exception(msg)
fname = fname[0]
houst = parse_xml(fname)
houst = houst.rename(
    columns={
        'SHORTNAME': 'TYPENAME',
        'DESC': 'TYPELONGNAME',
        'ID': 'HOUSETYPE'
    }
)
houst.head()

**Here's the main difference: two additional columns to store buildings.**

In [ ]:
fname = glob.glob('data/AS_ADDHOUSE_TYPES_*.XML')
if len(fname) != 1:
    msg = f'Please check file count for region {region} there are {len(fname)} files'
    logger.error(msg)
    raise Exception(msg)
fname = fname[0]
housta = parse_xml(fname)
housta = housta.rename(
    columns={
        'SHORTNAME': 'TYPENAME',
        'DESC': 'TYPELONGNAME',
        'ID': 'HOUSETYPE'
    }
)
housta.head()

In [ ]:
hous = hous.merge(
    houst[[
        'HOUSETYPE', 'TYPENAME', 'TYPELONGNAME'
    ]].drop_duplicates(),
    on='HOUSETYPE'
)
if 'HOUSETYPE1' in hous.columns:
    hous = hous.merge(
        housta[[
            'HOUSETYPE', 'TYPENAME', 'TYPELONGNAME'
        ]].rename(
            columns={
                'HOUSETYPE': 'HOUSETYPE1'
            }
        ).drop_duplicates(),
        on='HOUSETYPE1',
        how='left',
        suffixes=(None, '1')
    )
else:
    hous['HOUSETYPE1'] = np.nan
    hous['TYPELONGNAME1'] = np.nan
    hous['HOUSENUM1'] = np.nan
    hous['TYPENAME1'] = np.nan
if 'HOUSETYPE2' in hous.columns:
    hous = hous.merge(
        housta[[
            'HOUSETYPE', 'TYPENAME', 'TYPELONGNAME'
        ]].rename(
            columns={
                'HOUSETYPE': 'HOUSETYPE2'
            }
        ).drop_duplicates(),
        on='HOUSETYPE2',
        how='left',
        suffixes=(None, '2')
    )
else:
    hous['HOUSETYPE2'] = np.nan
    hous['TYPELONGNAME2'] = np.nan
    hous['HOUSENUM2'] = np.nan
    hous['TYPENAME2'] = np.nan
cleanup(houst)
cleanup(housta)
hous.head()

In [ ]:
hous['LEVEL'] = '10'
hous['LEVELNAME'] = 'Здание/Сооружение'
hous['NAME'] = hous[['TYPELONGNAME', 'HOUSENUM']].apply(
    lambda x: (
        x['TYPELONGNAME'].lower() + ' '
        if x['TYPELONGNAME'] == x['TYPELONGNAME']
        else ''
    ) + x['HOUSENUM'],
    axis=1
)
hous['NAME1'] = hous[['TYPELONGNAME1', 'HOUSENUM1']].apply(
    lambda x: (
        x['TYPELONGNAME1'].lower() + ' '
        if x['TYPELONGNAME1'] == x['TYPELONGNAME1']
        else ''
    ) + (
        x['HOUSENUM1']
        if x['HOUSENUM1'] == x['HOUSENUM1']
        else ''
    ),
    axis=1
)
hous['NAME2'] = hous[['TYPELONGNAME2', 'HOUSENUM2']].apply(
    lambda x: (
        x['TYPELONGNAME2'].lower() + ' '
        if x['TYPELONGNAME2'] == x['TYPELONGNAME2']
        else ''
    ) + (
        x['HOUSENUM2']
        if x['HOUSENUM2'] == x['HOUSENUM2']
        else ''
    ),
    axis=1
)
hous.head()

In [ ]:
fname = glob.glob(os.path.join(region, 'AS_HOUSES_PARAMS_*.XML'))
fname = fname[0]
hp = parse_xml(fname)
hp = hp[
    (
        hp['TYPEID'] == '7'
    ) & (
        hp['CHANGEIDEND'] == '0'
    )
]
hp = hp[
    hp.ENDDATE.apply(
        lambda x: datetime.datetime.strptime(
            x, '%Y-%m-%d'
        ) > datetime.datetime.fromtimestamp(
            time.time()
        )
    )
]
hp.head()

In [ ]:
hp = hp[['OBJECTID', 'VALUE']].rename(
    columns={'VALUE': 'OKTMO'}
).drop_duplicates().groupby(
    by='OBJECTID'
).agg(
    lambda x: x.to_list()
).to_dict('index')
hp

**Let's concat our address objects with house objects.**

In [ ]:
hadobj = pd.concat(
  [
      adobj[[
          'OBJECTID', 'OBJECTGUID', 'NAME', 'TYPENAME', 'LEVEL',
          'ISACTUAL', 'ISACTIVE', 'TYPELONGNAME', 'LEVELNAME'
      ]],
      hous[[
          'OBJECTID', 'OBJECTGUID', 'HOUSENUM', 'HOUSETYPE',
          'TYPENAME', 'TYPELONGNAME', 'HOUSENUM1', 'HOUSETYPE1',
          'TYPENAME1', 'TYPELONGNAME1', 'HOUSENUM2', 'HOUSETYPE2',
          'TYPENAME2', 'TYPELONGNAME2', 'ISACTUAL', 'ISACTIVE',
          'LEVEL', 'NAME', 'NAME1', 'NAME2', 'LEVELNAME'
      ]]
  ],
  sort=True,
  ignore_index=True
)
cleanup(adobj)
cleanup(hous)
hadobj.head()

## Read Administrative Relations File

**More exciting part is coming, I guess. The important thing is whether `PATH` column is included or not. Cause if it isn't, we suffer in attempts to restore paths ourselves.**

In [ ]:
fname = glob.glob(os.path.join(region, 'AS_MUN_HIERARCHY_*.XML'))
if len(fname) != 1:
    msg = f'Please check file count for region {region} there are {len(fname)} files'
    logger.error(msg)
    raise Exception(msg)
fname = fname[0]
adm = parse_xml(fname)
adm = adm[
    adm.ENDDATE.apply(
        lambda x: datetime.datetime.strptime(
            x, '%Y-%m-%d'
        ) > datetime.datetime.fromtimestamp(
            time.time()
        )
    )
]
chready = 'PATH' in adm.columns
cols = ['OBJECTID', 'PARENTOBJID'] + (['PATH'] if chready else [])
adm0 = adm[adm['ISACTIVE'] == '1'][cols].merge(
    hadobj[(hadobj['ISACTUAL'] == '1') & (hadobj['ISACTIVE'] == '1')],
    on='OBJECTID'
)
cleanup(adm)
adm0.head()

In [ ]:
chready

## Building Address Chains

**Option 1: PATHs included, just split them up nicely starting from house objects (level 10).**

In [ ]:
if chready:
    chains = [
        tuple(y for y in reversed(x.split('.')))
        for x in tqdm(adm0[adm0['LEVEL'] == '10']['PATH'])
    ]
    cleanup(adm0)
    print(len(chains), chains[:5])

**Option 2: Start the painful process of assembling address chains recursively.**

In [ ]:
if not chready:
    # get child-parent dictionary
    rdadm = get_adms(adm0)
    rdadm = {k: list(set(v)) for k, v in rdadm.items()}
    cleanup(adm0)
    print([(k, v) for k, v in rdadm.items()][:5])

In [ ]:
hadobjd = hadobj.set_index('OBJECTID').to_dict('index')
if not chready:
    # building chains recursively
    chains = [
        get_adms_rec_rev([x], rdadm, hp, adobjp)
        for x in tqdm(hadobj[hadobj['LEVEL'] == '10']['OBJECTID'].drop_duplicates())
    ]
    chains = [x for x in chains if x is not None]
# save and clean
hadobj.to_csv(f'{region}_hadobj.csv', index=False)
cleanup(hadobj)
[(k, v) for k, v in hadobjd.items()][:2]

**Sometimes there are multiple chains for one object. So let's try to fix that.**

In [ ]:
dfch = pd.DataFrame()
if not chready:
    odd_chains = [
        tuple(x)
        for x in chains
        if type(x[0]) == list
    ]
    odd_chains = [reduce_included(x) for x in odd_chains]
    odd_chains = [x for x, y in odd_chains if y] + [z for x, y in odd_chains if not y for z in x]
    chains = [
        tuple(x)
        for x in chains
        if type(x[0]) != list
    ] + odd_chains

dfch['chain'] = list(set(chains))
cleanup(chains)
dfch.head()

**Time to process chains in a fashionable manner.**

In [ ]:
dfch['levchain'] = [
    tuple([hadobjd[y]['LEVEL'] for y in x if y != '0' and y in hadobjd])
    for x in tqdm(dfch['chain'])
]
dat = [
    {
        m: l
        for m, l in zip(x, y)
    }
    for x, y in zip(dfch['levchain'], dfch['chain'])
]
for i in range(10, 0, -1):
    dfch[f'{i}'] = [
        d[f'{i}']
        if f'{i}' in d
        else None
        for d in dat
    ]
dfch.head()

**Now we'll attempt to structure all those chains, so that they fit into old-fashioned way of saying the address.**

In [ ]:
chl = list(set(dfch['levchain'].apply(lambda x: '-'.join(x))))
df = pd.DataFrame()
df['levchain'] = chl
for i in range(10, 0, -1):
    dat = [(f'{i}' in y.split('-')) * 1 for y in chl]
    df[f'{i}'] = dat
df.head()

In [ ]:
lst = df.apply(get_town, axis=1)
df['street'] = [x[0] for x in lst]
df['town'] = [x[1] for x in lst]
df['leftover'] = [x[2] for x in lst]
df['muni'] = [x[3] for x in lst]
df['levchain'] = df['levchain'].apply(lambda x: tuple(x.split('-')))
df.head()

**Now join chains and their structure together.**

In [ ]:
dfch = dfch.merge(df[['levchain', 'street', 'town', 'leftover', 'muni']], on='levchain')
dfch['region'] = region
dfch.head()

## Save

**Should be self-explanatory**

In [ ]:
dfch.to_csv(f'{region}_parsed_chains.csv', index=False)

# Read Result

**This part is __somewhat__ independent, meaning you can restart the runtime after successfully running all the previous cells.**

In [ ]:
# replace with your path
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
%cd "drive/MyDrive/gar_colab/garbdfias"

In [ ]:
import glob
import random
import numpy as np
import pandas as pd
from ast import literal_eval
from tqdm import tqdm

In [ ]:
regions = glob.glob('data/[0-9][0-9]')
region = regions[0]
region

In [ ]:
hadobj = pd.read_csv(f'{region}_hadobj.csv')
hadobj.head()

**Just some useful utilities for a dataframe. Notice how some columns are not of type 'string' anymore, so we'll have to fix that.**

In [ ]:
hadobj.info()

In [ ]:
hadobj.describe()

In [ ]:
for x in [
          'OBJECTID', 'HOUSENUM', 'HOUSENUM1',
          'HOUSENUM2', 'LEVEL', 'NAME',
          'TYPELONGNAME', 'TYPELONGNAME1', 'TYPELONGNAME2',
          'TYPENAME', 'TYPENAME1', 'TYPENAME2']:
    hadobj[x] = hadobj[x].apply(lambda y: str(y) if y == y else np.nan)
hadobj['OBJECTID'].values[0]

In [ ]:
dfch = pd.read_csv(f'{region}_parsed_chains.csv')
dfch.head()

In [ ]:
for x in ['chain', 'levchain', 'leftover', 'muni']:
    dfch[x] = [literal_eval(y) for y in tqdm(dfch[x])]
for x in range(1, 11, 1):
    dfch[f'{x}'] = dfch[f'{x}'].astype(str).apply(lambda y: y.split('.')[0] if y == y else y)
    dfch[f'{x}'] = dfch[f'{x}'].apply(lambda y: y if y != '' else np.nan)
for x in ['street', 'town', 'region']:
    dfch[x] = dfch[x].astype(str).apply(lambda y: y.split('.')[0] if y == y else y)
    dfch[x] = dfch[x].apply(lambda y: y if y != '' else np.nan)

In [ ]:
dfch.head()

In [ ]:
hadobjd = hadobj.set_index('OBJECTID').to_dict('index')
[(k, v) for k, v in hadobjd.items()][:2]

## Plaintext Addresses

**Let's just conctenate object names. It's not a good idea sometimes, just keep that in mind.**

In [ ]:
addresses = {x[0]: [] for x in tqdm(dfch.chain)}
for x in tqdm(dfch.chain):
    for y in x:
        if y != '0':
            tpln = hadobjd[y]['TYPELONGNAME']
            tpln1 = hadobjd[y]['TYPELONGNAME1']
            tpln2 = hadobjd[y]['TYPELONGNAME2']
            if tpln == tpln:
                addresses[x[0]].append(tpln)
            if hadobjd[y]['LEVEL'] != '10':
                addresses[x[0]].append(hadobjd[y]['NAME'])
            else:
                addresses[x[0]].append(hadobjd[y]['HOUSENUM'])
                if hadobjd[y]['HOUSENUM1'] == hadobjd[y]['HOUSENUM1']:
                    if tpln1 == tpln1:
                        addresses[x[0]].append(tpln1)
                    addresses[x[0]].append(hadobjd[y]['HOUSENUM1'])
                    if hadobjd[y]['HOUSENUM2'] == hadobjd[y]['HOUSENUM2']:
                        if tpln2 == tpln2:
                            addresses[x[0]].append(tpln2)
                        addresses[x[0]].append(hadobjd[y]['HOUSENUM2'])
addresses = {
    k: ' '.join(v)
    for k, v in addresses.items()
}
sample = random.sample(addresses.keys(), 10)
{s: addresses[s].lower() for s in sample}

## Structured Addresses

**Let's give our addresses more familiar structure and look at some confusing examples.**

In [ ]:
def get_struct_addr(x):
    if x['town'] != 'nan' and x[x['town']] in hadobjd:
        town = hadobjd[x[x['town']]]
        town = town['TYPELONGNAME'] + ' ' + town['NAME']
    else:
        town = ''
    if x['street'] != 'nan' and x[x['street']] in hadobjd:
        street = hadobjd[x[x['street']]]
        street = street['TYPELONGNAME'] + ' ' + street['NAME']
    else:
        street = ''
    house = hadobjd[x['10']]
    tpln = house['TYPELONGNAME'] + ' ' if house['TYPELONGNAME'] == house['TYPELONGNAME'] else ''
    tpln1 = house['TYPELONGNAME1'] + ' ' if house['TYPELONGNAME1'] == house['TYPELONGNAME1'] else ''
    tpln2 = house['TYPELONGNAME2'] + ' ' if house['TYPELONGNAME2'] == house['TYPELONGNAME2'] else ''
    house0 = tpln + house['HOUSENUM']
    house1 = ''
    house2 = ''
    
    if house['HOUSENUM1'] == house['HOUSENUM1']:
        house1 = tpln1 + house['HOUSENUM1']
    if house['HOUSENUM2'] == house['HOUSENUM2']:
        house2 = tpln2 + house['HOUSENUM2']
    leftover = []
    for y in x['leftover']:
        leftover.append(hadobjd[x[y]]['TYPELONGNAME'])
        leftover.append(hadobjd[x[y]]['NAME'])
    leftover = ' '.join(leftover)
    muni = []
    for y in x['muni']:
        muni.append(hadobjd[x[y]]['TYPELONGNAME'])
        muni.append(hadobjd[x[y]]['NAME'])
    muni = ' '.join(muni)
    return {
        'town': town,
        'street': street,
        'house': house0,
        'house1': house1,
        'house2': house2,
        'leftover': leftover,
        'muni': muni
    }

In [ ]:
dfch.head()

In [ ]:
struct_addresses = dfch.apply(get_struct_addr, axis=1)
struct_addresses[0]

In [ ]:
structdf = pd.DataFrame.from_records(struct_addresses)
structdf.sample(10)

In [ ]:
grouped = structdf.groupby(by=['town', 'street', 'house', 'house1', 'house2']).aggregate(lambda x: x.to_list())
grouped.head()

In [ ]:
grouped[(grouped['muni'].apply(lambda x: len(x) > 1))].sample(10)